In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ipywidgets as w
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import ez_torch as ez
import torch.nn as nn
import torchvision
from ez_torch.modules import Module, SpatialUVOffsetTransformer

In [5]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.feature_model = nn.Sequential(
            torchvision.models.resnet18(
                pretrained=False,
                progress=False,
                num_classes=32,
            ),
            nn.ReLU(),
        )
        self.transform = SpatialUVOffsetTransformer(
            32,
            uv_resolution_shape=(10, 10),
        )

    def criterion(self, y, y_target):
        return F.binary_cross_entropy(y, y_target)

    def forward(self, x):
        x_3d = x.repeat([1, 3, 1, 1])
        X_features = self.feature_model(x_3d)
        X_transformed = self.transform([X_features, x])
        return X_transformed
    
    def training_step(self, batch):
        pass

In [6]:
Model().ui

TrainableUI()

In [7]:
from ez_torch.ui import ui

In [8]:
view = ui.v(
    ui.h(
        ui.v(
            ui.label(value='LR'),
            ui.slider('lr', min=0.001, max=1, layout={'width': 'auto'}),
            ui.label(value='ITS'),
            ui.int('its', min=1, max=1000, default=100, layout={'width': 'auto'}),
            layout=w.Layout(**{'width': '25%', 'border': '2px solid #ccc'}),
        ),
        ui.v(
            ui.h(
                ui.progress('progress', description='IT [000:000]'),
                ui.button('start', description='Start'),
                ui.button('reset', description='Restart'),
            ),
            ui.line('loss', title='Loss', layout={'height': '350px'}),
            layout={'width': '100%', 'border': '2px solid #ccc'},
        ),
        layout={'border': '2px solid #ccc'},
    ),
    ui.output('out'),
    layout={'border': '4px solid #e6e6e6', 'paddnig': '10px'},
)

view

In [9]:
row = ui.h(ui.output('left'), ui.output('right'))
view.out.display(row)

In [10]:
n = 100
y = np.random.rand(n)
for i in range(1, n):

    view.loss.plot(y[:i])
    with row.left:
        plt.imshow(np.random.rand(10, 10))
        plt.show()
    with row.right:
        plt.imshow(np.random.rand(10, 10))
        plt.show()